# Hitchhiker's Guide to Hyperspace (.NET for Spark C#)
## An Indexing Subsystem for Apache Spark™

<img src="https://raw.githubusercontent.com/rapoth/hyperspace/master/docs/assets/images/hyperspace-small-banner.png" alt="Hyperspace Indexing Sub-System Logo" width="1000"/>

[Hyperspace](https://github.com/microsoft/hyperspace) introduces the ability for Apache Spark™ users to create indexes on their datasets (e.g., CSV, JSON, Parquet etc.) and leverage them for potential query and workload acceleration.

In this notebook, we highlight the basics of Hyperspace, emphasizing on its simplicity and show how it can be used by just anyone.

**Disclaimer**: Hyperspace helps accelerate your workloads/queries under two circumstances:

  1. Queries contain filters on predicates with high selectivity (e.g., you want to select 100 matching rows from a million candidate rows)
  2. Queries contain a join that requires heavy-shuffles (e.g., you want to join a 100 GB dataset with a 10 GB dataset)

You may want to carefully monitor your workloads and determine whether indexing is helping you on a case-by-case basis.

## Setup
To begin with, let's start a new Spark™ session. Since this notebook is a tutorial merely to illustrate what Hyperspace can offer, we will make a configuration change that allow us to highlight what Hyperspace is doing on small datasets. By default, Spark™ uses *broadcast join* to optimize join queries when the data size for one side of join is small (which is the case for the sample data we use in this tutorial). Therefore, we disable broadcast joins so that later when we run join queries, Spark™ uses *sort-merge* join. This is mainly to show how Hyperspace indexes would be used at scale for accelerating join queries.

The output of running the cell below shows a reference to the successfully created Spark™ session and prints out '-1' as the value for the modified join config which indicates that broadcast join is successfully disabled.

In [1]:
var sessionId = (new Random()).Next(10000000);
var dataPath = $"/hyperspace/data-{sessionId}";
var indexLocation = $"/hyperspace/indexes-{sessionId}";

// Please note that you DO NOT need to change this configuration in production.
// We store all indexes in the system folder within Synapse.
spark.Conf().Set("spark.hyperspace.system.path", indexLocation);

StatementMeta(medium, 21, 1, Finished, Available)

In [2]:
// Disable BroadcastHashJoin, so Spark™ will use standard SortMergeJoin. Currently hyperspace indexes utilize SortMergeJoin to speed up query.
spark.Conf().Set("spark.sql.autoBroadcastJoinThreshold", -1);

// Verify that BroadcastHashJoin is set correctly 
Console.WriteLine(spark.Conf().Get("spark.sql.autoBroadcastJoinThreshold"));

spark.Conf().Set("spark.hyperspace.explain.displayMode", "html")

StatementMeta(medium, 21, 2, Finished, Available)

-1


## Data Preparation

To prepare our environment, we will create sample data records and save them as parquet data files. While we use Parquet for illustration, you can use other formats such as CSV. In the subsequent cells, we will also demonstrate how you can create several Hyperspace indexes on this sample dataset and how one can make Spark™ use them when running queries. 

Our example records correspond to two datasets: *department* and *employee*. You should configure "empLocation" and "deptLocation" paths so that on the storage account they point to your desired location to save generated data files. 

The output of running below cell shows contents of our datasets as lists of triplets followed by references to dataFrames created to save the content of each dataset in our preferred location.

In [3]:
using Microsoft.Spark.Sql.Types;

// Sample department records
var departments = new List<GenericRow>()
{
    new GenericRow(new object[] {10, "Accounting", "New York"}),
    new GenericRow(new object[] {20, "Research", "Dallas"}),
    new GenericRow(new object[] {30, "Sales", "Chicago"}),
    new GenericRow(new object[] {40, "Operations", "Boston"})
};

// Sample employee records
var employees = new List<GenericRow>() {
      new GenericRow(new object[] {7369, "SMITH", 20}),
      new GenericRow(new object[] {7499, "ALLEN", 30}),
      new GenericRow(new object[] {7521, "WARD", 30}),
      new GenericRow(new object[] {7566, "JONES", 20}),
      new GenericRow(new object[] {7698, "BLAKE", 30}),
      new GenericRow(new object[] {7782, "CLARK", 10}),
      new GenericRow(new object[] {7788, "SCOTT", 20}),
      new GenericRow(new object[] {7839, "KING", 10}),
      new GenericRow(new object[] {7844, "TURNER", 30}),
      new GenericRow(new object[] {7876, "ADAMS", 20}),
      new GenericRow(new object[] {7900, "JAMES", 30}),
      new GenericRow(new object[] {7934, "MILLER", 10}),
      new GenericRow(new object[] {7902, "FORD", 20}),
      new GenericRow(new object[] {7654, "MARTIN", 30})
};

// Save sample data in the Parquet format
var departmentSchema = new StructType(new List<StructField>()
{
    new StructField("deptId", new IntegerType()),
    new StructField("deptName", new StringType()),
    new StructField("location", new StringType())
});
var employeeSchema = new StructType(new List<StructField>()
{
    new StructField("empId", new IntegerType()),
    new StructField("empName", new StringType()),
    new StructField("deptId", new IntegerType())
});

DataFrame empData = spark.CreateDataFrame(employees, employeeSchema); 
DataFrame deptData = spark.CreateDataFrame(departments, departmentSchema); 

string empLocation = $"{dataPath}/employees.parquet";
string deptLocation = $"{dataPath}/departments.parquet";
empData.Write().Mode("overwrite").Parquet(empLocation);
deptData.Write().Mode("overwrite").Parquet(deptLocation);

StatementMeta(medium, 21, 3, Finished, Available)

Let's verify the contents of parquet files we created above to make sure they contain expected records in correct format. We later use these data files to create Hyperspace indexes and run sample queries.

Running below cell, the output displays the rows in employee and department dataframes in a tabular form. There should be 14 employees and 4 departments, each matching with one of triplets we created in the previous cell.

In [4]:
// empLocation and deptLocation are the user defined locations above to save parquet files
DataFrame empDF = spark.Read().Parquet(empLocation);
DataFrame deptDF = spark.Read().Parquet(deptLocation);

// Verify the data is available and correct
empDF.Show();
deptDF.Show();

StatementMeta(medium, 21, 4, Finished, Available)

+-----+-------+------+
|empId|empName|deptId|
+-----+-------+------+
| 7844| TURNER|    30|
| 7934| MILLER|    10|
| 7654| MARTIN|    30|
| 7369|  SMITH|    20|
| 7499|  ALLEN|    30|
| 7566|  JONES|    20|
| 7698|  BLAKE|    30|
| 7782|  CLARK|    10|
| 7788|  SCOTT|    20|
| 7876|  ADAMS|    20|
| 7900|  JAMES|    30|
| 7521|   WARD|    30|
| 7839|   KING|    10|
| 7902|   FORD|    20|
+-----+-------+------+

+------+----------+--------+
|deptId|  deptName|location|
+------+----------+--------+
|    10|Accounting|New York|
|    40|Operations|  Boston|
|    20|  Research|  Dallas|
|    30|     Sales| Chicago|
+------+----------+--------+



## Hello Hyperspace Index!
Hyperspace lets users create indexes on records scanned from persisted data files. Once successfully created, an entry corresponding to the index is added to the Hyperspace's metadata. This metadata is later used by Apache Spark™'s Hyperspace-enabled optimizer during query processing to find and use proper indexes. 

Once indexes are created, users can perform several actions:
  - **Refresh** If the underlying data changes, users can refresh an existing index to capture that. 
  - **Delete** If the index is not needed, users can perform a soft-delete i.e., index is not physically deleted but is marked as 'deleted' so it is no longer used in your workloads.
  - **Vacuum** If an index is no longer required, users can vacuum it which forces a physical deletion of the index contents and associated metadata completely from Hyperspace's metadata.

Below sections show how such index management operations can be done in Hyperspace.

First, we need to import the required libraries and create an instance of Hyperspace. We later use this instance to invoke different Hyperspace APIs to create indexes on our sample data and modify those indexes.

Output of running below cell shows a reference to the created instance of Hyperspace.

In [5]:
// Create an instance of Hyperspace
using Microsoft.Spark.Extensions.Hyperspace;

Hyperspace hyperspace = new Hyperspace(spark);

StatementMeta(medium, 21, 5, Finished, Available)

### Create Indexes
To create a Hyperspace index, the user needs to provide 2 pieces of information:
* An Apache Spark™ DataFrame which references the data to be indexed.
* An index configuration object: IndexConfig, which specifies the *index name*, *indexed* and *included* columns of the index. 

As you might have noticed, in this notebook, we illustrate indexing using the [Covering Index](https://www.red-gate.com/simple-talk/sql/learn-sql-server/using-covering-indexes-to-improve-query-performance/), which are the default index in Hyperspace. In the future, we plan on adding support for other index types. 

We start by creating three Hyperspace indexes on our sample data: two indexes on the department dataset named "deptIndex1" and "deptIndex2", and one index on the employee dataset named 'empIndex'. 
For each index, we need a corresponding IndexConfig to capture the name along with columns lists for the indexed and included columns. Running below cell creates these indexConfigs and its output lists them.

**Note**: An *index column* is a column that appears in your filters or join conditions. An *included column* is a column that appears in your select/project.

For instance, in the following query:
```sql
SELECT X
FROM Table
WHERE Y = 2
```
X can be an *index column* and Y can be an *included column*.

In [6]:
// Create index configurations
using Microsoft.Spark.Extensions.Hyperspace.Index;

var empIndexConfig = new IndexConfig("empIndex", new string[] {"deptId"}, new string[] {"empName"});
var deptIndexConfig1 = new IndexConfig("deptIndex1", new string[] {"deptId"}, new string[] {"deptName"});
var deptIndexConfig2 = new IndexConfig("deptIndex2", new string[] {"location"}, new string[] {"deptName"});

StatementMeta(medium, 21, 6, Finished, Available)

Now, we create three indexes using our index configurations. For this purpose, we invoke "createIndex" command on our Hyperspace instance. This command requires an index configuration and the dataFrame containing rows to be indexed.
Running below cell creates three indexes.


In [7]:
// Create indexes from configurations
hyperspace.CreateIndex(empDF, empIndexConfig);
hyperspace.CreateIndex(deptDF, deptIndexConfig1);
hyperspace.CreateIndex(deptDF, deptIndexConfig2);

StatementMeta(medium, 21, 7, Finished, Available)

### List Indexes

Below code shows how a user can list all available indexes in a Hyperspace instance. It uses the `indexes` API which returns information about existing indexes as a Spark™'s DataFrame so you can perform additional operations. For instance, you can invoke valid operations on this DataFrame for checking its content or analyzing it further (for example filtering specific indexes or grouping them according to some desired property). 

Below cell uses DataFrame's `show` action to fully print the rows and show details of our indexes in a tabular form. For each index, we can see all the information Hyperspace has stored about it in its metadata. 

You will immediately notice the following:
  - `config.indexName`, `config.indexedColumns`, `config.includedColumns` are the fields that a user normally provides during index creation.
  - `status.status` indicates if the index is being actively used by the Spark's optimizer.
  - `dfSignature` is automatically generated by Hyperspace and is unique for each index. Hyperspace uses this signature internally to maintain the index and exploit it at query time. 
  
In the output below, all three indexes should have "ACTIVE" as status and their name, indexed columns, and included columns should match with what we defined in index configurations above.

In [8]:
hyperspace.Indexes().Show();

StatementMeta(medium, 21, 8, Finished, Available)

+----------+--------------+---------------+----------+--------------------+--------------------+------+
|      name|indexedColumns|includedColumns|numBuckets|              schema|       indexLocation| state|
+----------+--------------+---------------+----------+--------------------+--------------------+------+
|deptIndex1|      [deptId]|     [deptName]|       200|{"type":"struct",...|abfss://default@h...|ACTIVE|
|deptIndex2|    [location]|     [deptName]|       200|{"type":"struct",...|abfss://default@h...|ACTIVE|
|  empIndex|      [deptId]|      [empName]|       200|{"type":"struct",...|abfss://default@h...|ACTIVE|
+----------+--------------+---------------+----------+--------------------+--------------------+------+



### Delete Indexes
A user can drop an existing index by using the `deleteIndex` API and providing the index name. 

Index deletion is a **soft-delete** operation i.e., only the index's status in the Hyperspace metadata from is changed from "ACTIVE" to "DELETED". This will exclude the deleted index from any future query optimization and Hyperspace no longer picks that index for any query. However, index files for a deleted index still remain available (since it is a soft-delete), so if you accidentally deleted the index, you could still restore it.

The cell below deletes index with name "deptIndex2" and lists Hyperspace metadata after that. The output should be similar to above cell for "List Indexes" except for "deptIndex2" which now should have its status changed into "DELETED".

In [9]:
hyperspace.DeleteIndex("deptIndex2");

hyperspace.Indexes().Show();

StatementMeta(medium, 21, 9, Finished, Available)

+----------+--------------+---------------+----------+--------------------+--------------------+-------+
|      name|indexedColumns|includedColumns|numBuckets|              schema|       indexLocation|  state|
+----------+--------------+---------------+----------+--------------------+--------------------+-------+
|deptIndex1|      [deptId]|     [deptName]|       200|{"type":"struct",...|abfss://default@h...| ACTIVE|
|deptIndex2|    [location]|     [deptName]|       200|{"type":"struct",...|abfss://default@h...|DELETED|
|  empIndex|      [deptId]|      [empName]|       200|{"type":"struct",...|abfss://default@h...| ACTIVE|
+----------+--------------+---------------+----------+--------------------+--------------------+-------+



### Restore Indexes
A user can use the `restoreIndex` API to restore a deleted index. This will bring back the latest version of index into ACTIVE status and makes it usable again for queries. 

The cell below shows an example of `restoreIndex` API. We delete "deptIndex1" and restore it. The output shows "deptIndex1" first went into the "DELETED" status after invoking "deleteIndex" command and came back to the "ACTIVE" status after calling "restoreIndex".

In [10]:
hyperspace.DeleteIndex("deptIndex1");

hyperspace.Indexes().Show();

hyperspace.RestoreIndex("deptIndex1");

hyperspace.Indexes().Show();

StatementMeta(medium, 21, 10, Finished, Available)

+----------+--------------+---------------+----------+--------------------+--------------------+-------+
|      name|indexedColumns|includedColumns|numBuckets|              schema|       indexLocation|  state|
+----------+--------------+---------------+----------+--------------------+--------------------+-------+
|deptIndex1|      [deptId]|     [deptName]|       200|{"type":"struct",...|abfss://default@h...|DELETED|
|deptIndex2|    [location]|     [deptName]|       200|{"type":"struct",...|abfss://default@h...|DELETED|
|  empIndex|      [deptId]|      [empName]|       200|{"type":"struct",...|abfss://default@h...| ACTIVE|
+----------+--------------+---------------+----------+--------------------+--------------------+-------+

+----------+--------------+---------------+----------+--------------------+--------------------+-------+
|      name|indexedColumns|includedColumns|numBuckets|              schema|       indexLocation|  state|
+----------+--------------+---------------+----------+

### Vacuum Indexes
The user can perform a **hard-delete** i.e., fully remove files and the metadata entry for a deleted index using the `vacuumIndex` API. Once done, this action is **irreversible** as it physically deletes all the index files associated with the index.

The cell below vacuums the "deptIndex2" index and shows Hyperspace metadata after vaccuming. You should see metadata entries for two indexes "deptIndex1" and "empIndex" both with "ACTIVE" status and no entry for "deptIndex2".

In [11]:
hyperspace.VacuumIndex("deptIndex2");

hyperspace.Indexes().Show();

StatementMeta(medium, 21, 11, Finished, Available)

+----------+--------------+---------------+----------+--------------------+--------------------+------+
|      name|indexedColumns|includedColumns|numBuckets|              schema|       indexLocation| state|
+----------+--------------+---------------+----------+--------------------+--------------------+------+
|deptIndex1|      [deptId]|     [deptName]|       200|{"type":"struct",...|abfss://default@h...|ACTIVE|
|  empIndex|      [deptId]|      [empName]|       200|{"type":"struct",...|abfss://default@h...|ACTIVE|
+----------+--------------+---------------+----------+--------------------+--------------------+------+



## Enable/Disable Hyperspace

Hyperspace provides APIs to enable or disable index usage with Spark™.

  - By using `enableHyperspace` API, Hyperspace optimization rules become visible to the Apache Spark™ optimizer and it will exploit existing Hyperspace indexes to optimize user queries.
  - By using `disableHyperspace` command, Hyperspace rules no longer apply during query optimization. You should note that disabling Hyperspace has no impact on created indexes as they remain intact.

Below cell shows how you can use these commands to enable or disable hyperspace. The output simply shows a reference to the existing Spark™ session whose configuration is updated.

In [12]:
// Enable Hyperspace
spark.EnableHyperspace();

// Disable Hyperspace
spark.DisableHyperspace();

StatementMeta(medium, 21, 12, Finished, Available)

## Index Usage
In order to make Spark™ use Hyperspace indexes during query processing, the user needs to make sure that Hyperspace is enabled. 

The cell below enables Hyperspace and creates two DataFrames containing our sample data records which we use for running example queries. For each DataFrame, a few sample rows are printed.

In [13]:
// Enable Hyperspace
spark.EnableHyperspace();

DataFrame empDFrame = spark.Read().Parquet(empLocation);
DataFrame deptDFrame = spark.Read().Parquet(deptLocation);

empDFrame.Show(5);
deptDFrame.Show(5);

StatementMeta(medium, 21, 13, Finished, Available)

+-----+-------+------+
|empId|empName|deptId|
+-----+-------+------+
| 7844| TURNER|    30|
| 7934| MILLER|    10|
| 7654| MARTIN|    30|
| 7369|  SMITH|    20|
| 7499|  ALLEN|    30|
+-----+-------+------+
only showing top 5 rows

+------+----------+--------+
|deptId|  deptName|location|
+------+----------+--------+
|    10|Accounting|New York|
|    40|Operations|  Boston|
|    20|  Research|  Dallas|
|    30|     Sales| Chicago|
+------+----------+--------+



# Hyperspace's Index Types

Currently, Hyperspace can exploit indexes for two groups of queries: 
* Selection queries with lookup or range selection filtering predicates.
* Join queries with an equality join predicate (i.e. Equi-joins).

## Indexes for Accelerating Filters

Our first example query does a lookup on department records (see below cell). In SQL, this query looks as follows:

```sql
SELECT deptName 
FROM departments
WHERE deptId = 20
```

The output of running the cell below shows: 
- query result, which is a single department name.
- query plan that Spark™ used to run the query. 

In the query plan, the "FileScan" operator at the bottom of the plan shows the datasource where the records were read from. The location of this file indicates the path to the latest version of the "deptIndex1" index. This shows  that according to the query and using Hyperspace optimization rules, Spark™ decided to exploit the proper index at runtime.


In [14]:
// Filter with equality predicate
DataFrame eqFilter = deptDFrame.Filter("deptId = 20").Select("deptName");
eqFilter.Show();

hyperspace.Explain(eqFilter, true, input => DisplayHTML(input));

StatementMeta(medium, 21, 14, Finished, Available)

+--------+
|deptName|
+--------+
|Research|
+--------+



============================================================= Plan with indexes: ============================================================= Project [deptName#513] +- Filter (isnotnull(deptId#512) && (deptId#512 = 20)) +- FileScan Hyperspace(Type: CI, Name: deptIndex1, LogVersion: 5) [deptId#512,deptName#513] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://default@honghaigen2.dfs.core.windows.net/hyperspace/indexes-6785343/dep..., PartitionFilters: [], PushedFilters: [IsNotNull(deptId), EqualTo(deptId,20)], ReadSchema: struct ============================================================= Plan without indexes: ============================================================= Project [deptName#513] +- Filter (isnotnull(deptId#512) && (deptId#512 = 20)) +- FileScan parquet [deptId#512,deptName#513] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://default@honghaigen2.dfs.core.windows.net/hyperspace/data-6785343/depart..., PartitionFilters: [], PushedFilters: [IsNotNull(deptId), EqualTo(deptId,20)], ReadSchema: struct ============================================================= Indexes used: ============================================================= deptIndex1:abfss://default@honghaigen2.dfs.core.windows.net/hyperspace/indexes-6785343/deptIndex1/v__=0 ============================================================= Physical operator stats: ============================================================= +-----------------------------------------------------------+-------------------+------------------+----------+ | Physical Operator|Hyperspace Disabled|Hyperspace Enabled|Difference| +-----------------------------------------------------------+-------------------+------------------+----------+ |*Scan Hyperspace(Type: CI, Name: deptIndex1, LogVersion: 5)| 0| 1| 1| | *Scan parquet| 1| 0| -1| | Filter| 1| 1| 0| | Project| 1| 1| 0| | WholeStageCodegen| 1| 1| 0| +-----------------------------------------------------------+-------------------+------------------+----------+

Our second example is a range selection query on department records. In SQL, this query looks as follows:

```sql
SELECT deptName 
FROM departments
WHERE deptId > 20
```
Similar to our first example, the output of the cell below shows the query results (names of two departments) and the query plan. The location of data file in the FileScan operator shows that 'deptIndex1" was used to run the query.   


In [15]:
// Filter with range selection predicate
DataFrame rangeFilter = deptDFrame.Filter("deptId > 20").Select("deptName");
rangeFilter.Show();

hyperspace.Explain(rangeFilter, true, input => DisplayHTML(input));

StatementMeta(medium, 21, 15, Finished, Available)

+----------+
|  deptName|
+----------+
|Operations|
|     Sales|
+----------+



============================================================= Plan with indexes: ============================================================= Project [deptName#513] +- Filter (isnotnull(deptId#512) && (deptId#512 > 20)) +- FileScan Hyperspace(Type: CI, Name: deptIndex1, LogVersion: 5) [deptId#512,deptName#513] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://default@honghaigen2.dfs.core.windows.net/hyperspace/indexes-6785343/dep..., PartitionFilters: [], PushedFilters: [IsNotNull(deptId), GreaterThan(deptId,20)], ReadSchema: struct ============================================================= Plan without indexes: ============================================================= Project [deptName#513] +- Filter (isnotnull(deptId#512) && (deptId#512 > 20)) +- FileScan parquet [deptId#512,deptName#513] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://default@honghaigen2.dfs.core.windows.net/hyperspace/data-6785343/depart..., PartitionFilters: [], PushedFilters: [IsNotNull(deptId), GreaterThan(deptId,20)], ReadSchema: struct ============================================================= Indexes used: ============================================================= deptIndex1:abfss://default@honghaigen2.dfs.core.windows.net/hyperspace/indexes-6785343/deptIndex1/v__=0 ============================================================= Physical operator stats: ============================================================= +-----------------------------------------------------------+-------------------+------------------+----------+ | Physical Operator|Hyperspace Disabled|Hyperspace Enabled|Difference| +-----------------------------------------------------------+-------------------+------------------+----------+ |*Scan Hyperspace(Type: CI, Name: deptIndex1, LogVersion: 5)| 0| 1| 1| | *Scan parquet| 1| 0| -1| | Filter| 1| 1| 0| | Project| 1| 1| 0| | WholeStageCodegen| 1| 1| 0| +-----------------------------------------------------------+-------------------+------------------+----------+

Our third example is a query joining department and employee records on the department id. The equivalent SQL statement is shown below:

```sql
SELECT employees.deptId, empName, departments.deptId, deptName
FROM   employees, departments 
WHERE  employees.deptId = departments.deptId
```

The output of running the cell below shows the query results which are the names of 14 employees and the name of department each employee works in. The query plan is also included in the output. Notice how the file locations for two FileScan operators shows that Spark™ used "empIndex" and "deptIndex1" indexes to run the query.   


In [16]:
// Join
DataFrame eqJoin =
      empDFrame
      .Join(deptDFrame, empDFrame.Col("deptId") == deptDFrame.Col("deptId"))
      .Select(empDFrame.Col("empName"), deptDFrame.Col("deptName"));

eqJoin.Show();

hyperspace.Explain(eqJoin, true, input => DisplayHTML(input));

StatementMeta(medium, 21, 16, Finished, Available)

+-------+----------+
|empName|  deptName|
+-------+----------+
|  SCOTT|  Research|
|  SMITH|  Research|
|  JONES|  Research|
|  ADAMS|  Research|
|   FORD|  Research|
| MILLER|Accounting|
|  CLARK|Accounting|
|   KING|Accounting|
| TURNER|     Sales|
| MARTIN|     Sales|
|  ALLEN|     Sales|
|  BLAKE|     Sales|
|  JAMES|     Sales|
|   WARD|     Sales|
+-------+----------+



============================================================= Plan with indexes: ============================================================= Project [empName#507, deptName#513] +- SortMergeJoin [deptId#508], [deptId#512], Inner :- *(1) Project [empName#507, deptId#508] : +- *(1) Filter isnotnull(deptId#508) : +- *(1) FileScan Hyperspace(Type: CI, Name: empIndex, LogVersion: 1) [deptId#508,empName#507] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://default@honghaigen2.dfs.core.windows.net/hyperspace/indexes-6785343/emp..., PartitionFilters: [], PushedFilters: [IsNotNull(deptId)], ReadSchema: struct , SelectedBucketsCount: 200 out of 200 +- *(2) Project [deptId#512, deptName#513] +- *(2) Filter isnotnull(deptId#512) +- *(2) FileScan Hyperspace(Type: CI, Name: deptIndex1, LogVersion: 5) [deptId#512,deptName#513] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://default@honghaigen2.dfs.core.windows.net/hyperspace/indexes-6785343/dep..., PartitionFilters: [], PushedFilters: [IsNotNull(deptId)], ReadSchema: struct , SelectedBucketsCount: 200 out of 200 ============================================================= Plan without indexes: ============================================================= Project [empName#507, deptName#513] +- SortMergeJoin [deptId#508], [deptId#512], Inner :- *(2) Sort [deptId#508 ASC NULLS FIRST], false, 0 : +- Exchange hashpartitioning(deptId#508, 200), [id=#183] : +- *(1) Project [empName#507, deptId#508] : +- *(1) Filter isnotnull(deptId#508) : +- *(1) FileScan parquet [empName#507,deptId#508] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://default@honghaigen2.dfs.core.windows.net/hyperspace/data-6785343/employ..., PartitionFilters: [], PushedFilters: [IsNotNull(deptId)], ReadSchema: struct +- *(4) Sort [deptId#512 ASC NULLS FIRST], false, 0 +- Exchange hashpartitioning(deptId#512, 200), [id=#189] +- *(3) Project [deptId#512, deptName#513] +- *(3) Filter isnotnull(deptId#512) +- *(3) FileScan parquet [deptId#512,deptName#513] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://default@honghaigen2.dfs.core.windows.net/hyperspace/data-6785343/depart..., PartitionFilters: [], PushedFilters: [IsNotNull(deptId)], ReadSchema: struct ============================================================= Indexes used: ============================================================= deptIndex1:abfss://default@honghaigen2.dfs.core.windows.net/hyperspace/indexes-6785343/deptIndex1/v__=0 empIndex:abfss://default@honghaigen2.dfs.core.windows.net/hyperspace/indexes-6785343/empIndex/v__=0 ============================================================= Physical operator stats: ============================================================= +-----------------------------------------------------------+-------------------+------------------+----------+ | Physical Operator|Hyperspace Disabled|Hyperspace Enabled|Difference| +-----------------------------------------------------------+-------------------+------------------+----------+ | *InputAdapter| 4| 2| -2| |*Scan Hyperspace(Type: CI, Name: deptIndex1, LogVersion: 5)| 0| 1| 1| | *Scan Hyperspace(Type: CI, Name: empIndex, LogVersion: 1)| 0| 1| 1| | *Scan parquet| 2| 0| -2| | *ShuffleExchange| 2| 0| -2| | *Sort| 2| 0| -2| | *WholeStageCodegen| 5| 3| -2| | Filter| 2| 2| 0| | Project| 3| 3| 0| | SortMergeJoin| 1| 1| 0| +-----------------------------------------------------------+-------------------+------------------+----------+

## Support for SQL Semantics

The index usage is transparent to whether the user uses DataFrame API or Spark™ SQL. The following example shows the same join example as before but using Spark SQL, showing the use of indexes if applicable.

In [17]:
empDFrame.CreateOrReplaceTempView("EMP");
deptDFrame.CreateOrReplaceTempView("DEPT");

var joinQuery = spark.Sql("SELECT EMP.empName, DEPT.deptName FROM EMP, DEPT WHERE EMP.deptId = DEPT.deptId");

joinQuery.Show();
hyperspace.Explain(joinQuery, true, input => DisplayHTML(input));

StatementMeta(medium, 21, 17, Finished, Available)

+-------+----------+
|empName|  deptName|
+-------+----------+
|  SCOTT|  Research|
|  SMITH|  Research|
|  JONES|  Research|
|  ADAMS|  Research|
|   FORD|  Research|
| MILLER|Accounting|
|  CLARK|Accounting|
|   KING|Accounting|
| TURNER|     Sales|
| MARTIN|     Sales|
|  ALLEN|     Sales|
|  BLAKE|     Sales|
|  JAMES|     Sales|
|   WARD|     Sales|
+-------+----------+



============================================================= Plan with indexes: ============================================================= Project [empName#507, deptName#513] +- SortMergeJoin [deptId#508], [deptId#512], Inner :- *(1) Project [empName#507, deptId#508] : +- *(1) Filter isnotnull(deptId#508) : +- *(1) FileScan Hyperspace(Type: CI, Name: empIndex, LogVersion: 1) [deptId#508,empName#507] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://default@honghaigen2.dfs.core.windows.net/hyperspace/indexes-6785343/emp..., PartitionFilters: [], PushedFilters: [IsNotNull(deptId)], ReadSchema: struct , SelectedBucketsCount: 200 out of 200 +- *(2) Project [deptId#512, deptName#513] +- *(2) Filter isnotnull(deptId#512) +- *(2) FileScan Hyperspace(Type: CI, Name: deptIndex1, LogVersion: 5) [deptId#512,deptName#513] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://default@honghaigen2.dfs.core.windows.net/hyperspace/indexes-6785343/dep..., PartitionFilters: [], PushedFilters: [IsNotNull(deptId)], ReadSchema: struct , SelectedBucketsCount: 200 out of 200 ============================================================= Plan without indexes: ============================================================= Project [empName#507, deptName#513] +- SortMergeJoin [deptId#508], [deptId#512], Inner :- *(2) Sort [deptId#508 ASC NULLS FIRST], false, 0 : +- Exchange hashpartitioning(deptId#508, 200), [id=#279] : +- *(1) Project [empName#507, deptId#508] : +- *(1) Filter isnotnull(deptId#508) : +- *(1) FileScan parquet [empName#507,deptId#508] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://default@honghaigen2.dfs.core.windows.net/hyperspace/data-6785343/employ..., PartitionFilters: [], PushedFilters: [IsNotNull(deptId)], ReadSchema: struct +- *(4) Sort [deptId#512 ASC NULLS FIRST], false, 0 +- Exchange hashpartitioning(deptId#512, 200), [id=#285] +- *(3) Project [deptId#512, deptName#513] +- *(3) Filter isnotnull(deptId#512) +- *(3) FileScan parquet [deptId#512,deptName#513] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://default@honghaigen2.dfs.core.windows.net/hyperspace/data-6785343/depart..., PartitionFilters: [], PushedFilters: [IsNotNull(deptId)], ReadSchema: struct ============================================================= Indexes used: ============================================================= deptIndex1:abfss://default@honghaigen2.dfs.core.windows.net/hyperspace/indexes-6785343/deptIndex1/v__=0 empIndex:abfss://default@honghaigen2.dfs.core.windows.net/hyperspace/indexes-6785343/empIndex/v__=0 ============================================================= Physical operator stats: ============================================================= +-----------------------------------------------------------+-------------------+------------------+----------+ | Physical Operator|Hyperspace Disabled|Hyperspace Enabled|Difference| +-----------------------------------------------------------+-------------------+------------------+----------+ | *InputAdapter| 4| 2| -2| |*Scan Hyperspace(Type: CI, Name: deptIndex1, LogVersion: 5)| 0| 1| 1| | *Scan Hyperspace(Type: CI, Name: empIndex, LogVersion: 1)| 0| 1| 1| | *Scan parquet| 2| 0| -2| | *ShuffleExchange| 2| 0| -2| | *Sort| 2| 0| -2| | *WholeStageCodegen| 5| 3| -2| | Filter| 2| 2| 0| | Project| 3| 3| 0| | SortMergeJoin| 1| 1| 0| +-----------------------------------------------------------+-------------------+------------------+----------+

## Explain API

So far, you might have observed we have been using the explain API provided by Hyperspace. The `explain` API from Hyperspace is very similar to Spark's `df.explain` API but allows users to compare their original plan vs the updated index-dependent plan before running their query. You have an option to choose from html/plaintext/console mode to display the command output. 

The following cell shows an example with HTML. The highlighted section represents the difference between original and updated plans along with the indexes being used.

In [18]:
spark.Conf().Set("spark.hyperspace.explain.displayMode", "html");
spark.Conf().Set("spark.hyperspace.explain.displayMode.highlight.beginTag", "<b style=\"background:LightGreen\">");
spark.Conf().Set("spark.hyperspace.explain.displayMode.highlight.endTag", "</b>");

hyperspace.Explain(eqJoin, true, input => DisplayHTML(input));

StatementMeta(medium, 21, 18, Finished, Available)

============================================================= Plan with indexes: ============================================================= Project [empName#507, deptName#513] +- SortMergeJoin [deptId#508], [deptId#512], Inner :- *(1) Project [empName#507, deptId#508] : +- *(1) Filter isnotnull(deptId#508) : +- *(1) FileScan Hyperspace(Type: CI, Name: empIndex, LogVersion: 1) [deptId#508,empName#507] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://default@honghaigen2.dfs.core.windows.net/hyperspace/indexes-6785343/emp..., PartitionFilters: [], PushedFilters: [IsNotNull(deptId)], ReadSchema: struct , SelectedBucketsCount: 200 out of 200 +- *(2) Project [deptId#512, deptName#513] +- *(2) Filter isnotnull(deptId#512) +- *(2) FileScan Hyperspace(Type: CI, Name: deptIndex1, LogVersion: 5) [deptId#512,deptName#513] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://default@honghaigen2.dfs.core.windows.net/hyperspace/indexes-6785343/dep..., PartitionFilters: [], PushedFilters: [IsNotNull(deptId)], ReadSchema: struct , SelectedBucketsCount: 200 out of 200 ============================================================= Plan without indexes: ============================================================= Project [empName#507, deptName#513] +- SortMergeJoin [deptId#508], [deptId#512], Inner :- *(2) Sort [deptId#508 ASC NULLS FIRST], false, 0 : +- Exchange hashpartitioning(deptId#508, 200), [id=#345] : +- *(1) Project [empName#507, deptId#508] : +- *(1) Filter isnotnull(deptId#508) : +- *(1) FileScan parquet [empName#507,deptId#508] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://default@honghaigen2.dfs.core.windows.net/hyperspace/data-6785343/employ..., PartitionFilters: [], PushedFilters: [IsNotNull(deptId)], ReadSchema: struct +- *(4) Sort [deptId#512 ASC NULLS FIRST], false, 0 +- Exchange hashpartitioning(deptId#512, 200), [id=#351] +- *(3) Project [deptId#512, deptName#513] +- *(3) Filter isnotnull(deptId#512) +- *(3) FileScan parquet [deptId#512,deptName#513] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://default@honghaigen2.dfs.core.windows.net/hyperspace/data-6785343/depart..., PartitionFilters: [], PushedFilters: [IsNotNull(deptId)], ReadSchema: struct ============================================================= Indexes used: ============================================================= deptIndex1:abfss://default@honghaigen2.dfs.core.windows.net/hyperspace/indexes-6785343/deptIndex1/v__=0 empIndex:abfss://default@honghaigen2.dfs.core.windows.net/hyperspace/indexes-6785343/empIndex/v__=0 ============================================================= Physical operator stats: ============================================================= +-----------------------------------------------------------+-------------------+------------------+----------+ | Physical Operator|Hyperspace Disabled|Hyperspace Enabled|Difference| +-----------------------------------------------------------+-------------------+------------------+----------+ | *InputAdapter| 4| 2| -2| |*Scan Hyperspace(Type: CI, Name: deptIndex1, LogVersion: 5)| 0| 1| 1| | *Scan Hyperspace(Type: CI, Name: empIndex, LogVersion: 1)| 0| 1| 1| | *Scan parquet| 2| 0| -2| | *ShuffleExchange| 2| 0| -2| | *Sort| 2| 0| -2| | *WholeStageCodegen| 5| 3| -2| | Filter| 2| 2| 0| | Project| 3| 3| 0| | SortMergeJoin| 1| 1| 0| +-----------------------------------------------------------+-------------------+------------------+----------+

## Refresh Indexes
If the original data on which an index was created changes, then the index will no longer capture the latest state of data and hence will not be used by Hyperspace to provide any acceleration. The user can refresh such a stale index using the `refreshIndex` API. This causes the index to be fully rebuilt and updates it according to the latest data records.
    
    Spoiler alert: if you are worried about fully rebuilding your index every time your data changes, don't worry! We will show you how to *incrementally refresh* your index in subsequent cells below.

The two cells below show an example for this scenario:
- First cell adds two more departments to the original departments data. It reads and prints list of departments to verify new departments are added correctly. The output shows 6 departments in total: four old ones and two new. Invoking "refreshIndex" updates "deptIndex1" so index captures new departments.
- Second cell runs our range selection query example. The results should now contain four departments: two are the ones, seen before when we ran the query above, and two are the new departments we just added.

In [19]:
var extraDepartments = new List<GenericRow>()
{
    new GenericRow(new object[] {50, "Inovation", "Seattle"}),
    new GenericRow(new object[] {60, "Human Resources", "San Francisco"})
};
	  
DataFrame extraDeptData = spark.CreateDataFrame(extraDepartments, departmentSchema);
extraDeptData.Write().Mode("Append").Parquet(deptLocation);

DataFrame deptDFrameUpdated = spark.Read().Parquet(deptLocation);

deptDFrameUpdated.Show(10);

hyperspace.RefreshIndex("deptIndex1");

StatementMeta(medium, 21, 19, Finished, Available)

+------+---------------+-------------+
|deptId|       deptName|     location|
+------+---------------+-------------+
|    60|Human Resources|San Francisco|
|    10|     Accounting|     New York|
|    50|      Inovation|      Seattle|
|    40|     Operations|       Boston|
|    20|       Research|       Dallas|
|    30|          Sales|      Chicago|
+------+---------------+-------------+



In [20]:
DataFrame newRangeFilter = deptDFrameUpdated.Filter("deptId > 20").Select("deptName");
newRangeFilter.Show();

hyperspace.Explain(newRangeFilter, true, input => DisplayHTML(input));

StatementMeta(medium, 21, 20, Finished, Available)

+---------------+
|       deptName|
+---------------+
|Human Resources|
|      Inovation|
|     Operations|
|          Sales|
+---------------+



============================================================= Plan with indexes: ============================================================= Project [deptName#1083] +- Filter (isnotnull(deptId#1082) && (deptId#1082 > 20)) +- FileScan Hyperspace(Type: CI, Name: deptIndex1, LogVersion: 7) [deptId#1082,deptName#1083] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://default@honghaigen2.dfs.core.windows.net/hyperspace/indexes-6785343/dep..., PartitionFilters: [], PushedFilters: [IsNotNull(deptId), GreaterThan(deptId,20)], ReadSchema: struct ============================================================= Plan without indexes: ============================================================= Project [deptName#1083] +- Filter (isnotnull(deptId#1082) && (deptId#1082 > 20)) +- FileScan parquet [deptId#1082,deptName#1083] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://default@honghaigen2.dfs.core.windows.net/hyperspace/data-6785343/depart..., PartitionFilters: [], PushedFilters: [IsNotNull(deptId), GreaterThan(deptId,20)], ReadSchema: struct ============================================================= Indexes used: ============================================================= deptIndex1:abfss://default@honghaigen2.dfs.core.windows.net/hyperspace/indexes-6785343/deptIndex1/v__=1 ============================================================= Physical operator stats: ============================================================= +-----------------------------------------------------------+-------------------+------------------+----------+ | Physical Operator|Hyperspace Disabled|Hyperspace Enabled|Difference| +-----------------------------------------------------------+-------------------+------------------+----------+ |*Scan Hyperspace(Type: CI, Name: deptIndex1, LogVersion: 7)| 0| 1| 1| | *Scan parquet| 1| 0| -1| | Filter| 1| 1| 0| | Project| 1| 1| 0| | WholeStageCodegen| 1| 1| 0| +-----------------------------------------------------------+-------------------+------------------+----------+

## Hybrid Scan for Mutable Datasets

Often times, if your underlying source data had some new files appended or existing files deleted, your index will get stale and Hyperspace decides not to use it. However, there are times where you just want to use the index without having to refresh it everytime. There could be multiple reasons for doing so:

  1. You do not want to continuosly refresh your index but instead want to do it periodically since you understand your workloads the best.  
  2. You added/removed only a few files and do not want to wait for yet another refresh job to finish. 

To allow you to still use a stale index, Hyperspace introduces **Hybrid Scan**, a novel technique that allows users to utilize outdated or stale indexes (e.g., the underlying source data had some new files appended or existing files deleted), without refreshing indexes. 

To achieve this, when you set the appropriate configuration to enable Hybrid Scan, Hyperspace modifies the query plan to leverage the changes as following:
- Appended files can be merged to index data by using **`Union` or `BucketUnion` (for join)**. Shuffling appended data can also be applied before merging, if needed.
- Deleted files can be handled by injecting `Filter-NOT-IN` condition on **lineage column** of index data, so that the indexed rows from the deleted files can be excluded at query time. 

You can check the transformation of the query plan in below examples.

    Note: Hybrid scan is only supported for non-partitioned data. Support for partitioned data is currently being worked upon.

### Hybrid Scan for appended files - non-partitioned data

Non-partitioned data is used in below example. In this example, we expect Join index can be used for the query and `BucketUnion` is introduced for appended files.

In [21]:
// GENERATE TEST DATA
using Microsoft.Spark.Sql.Types;

var products = new List<GenericRow>() {
    new GenericRow(new object[] {"orange", 3, "2020-10-01"}),
    new GenericRow(new object[] {"banana", 1, "2020-10-01"}),
    new GenericRow(new object[] {"carrot", 5, "2020-10-02"}),
    new GenericRow(new object[] {"beetroot", 12, "2020-10-02"}),
    new GenericRow(new object[] {"orange", 2, "2020-10-03"}),
    new GenericRow(new object[] {"banana", 11, "2020-10-03"}),
    new GenericRow(new object[] {"carrot", 3, "2020-10-03"}),
    new GenericRow(new object[] {"beetroot", 2, "2020-10-04"}),
    new GenericRow(new object[] {"cucumber", 7, "2020-10-05"}),
    new GenericRow(new object[] {"pepper", 20, "2020-10-06"})
};
var productsSchema = new StructType(new List<StructField>()
{
    new StructField("name", new StringType()),
    new StructField("qty", new IntegerType()),
    new StructField("date", new StringType())
});

DataFrame testData = spark.CreateDataFrame(products, productsSchema); 
string testDataLocation = $"{dataPath}/productTable";
testData.Write().Mode("overwrite").Parquet(testDataLocation);

StatementMeta(medium, 21, 21, Finished, Available)

In [22]:
// CREATE INDEX
DataFrame testDF = spark.Read().Parquet(testDataLocation);
var productIndex2Config = new IndexConfig("productIndex", new string[] {"name"}, new string[] {"date", "qty"});
hyperspace.CreateIndex(testDF, productIndex2Config);

StatementMeta(medium, 21, 22, Finished, Available)

In [23]:
DataFrame filter1 = testDF.Filter("name = 'banana'");
DataFrame filter2 = testDF.Filter("qty > 10");
DataFrame query = filter1.Join(filter2, filter1.Col("name") == filter2.Col("name"));

query.Show();

hyperspace.Explain(query, true, input => DisplayHTML(input));

StatementMeta(medium, 21, 23, Finished, Available)

+------+---+----------+------+---+----------+
|  name|qty|      date|  name|qty|      date|
+------+---+----------+------+---+----------+
|banana| 11|2020-10-03|banana| 11|2020-10-03|
|banana|  1|2020-10-01|banana| 11|2020-10-03|
+------+---+----------+------+---+----------+



============================================================= Plan with indexes: ============================================================= SortMergeJoin [name#1225], [name#1249], Inner :- *(1) Project [name#1225, qty#1226, date#1227] : +- *(1) Filter (isnotnull(name#1225) && (name#1225 = banana)) : +- *(1) FileScan Hyperspace(Type: CI, Name: productIndex, LogVersion: 1) [name#1225,date#1227,qty#1226] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://default@honghaigen2.dfs.core.windows.net/hyperspace/indexes-6785343/pro..., PartitionFilters: [], PushedFilters: [IsNotNull(name), EqualTo(name,banana)], ReadSchema: struct , SelectedBucketsCount: 1 out of 200 +- *(2) Project [name#1249, qty#1250, date#1251] +- *(2) Filter (((isnotnull(qty#1250) && (qty#1250 > 10)) && isnotnull(name#1249)) && (name#1249 = banana)) +- *(2) FileScan Hyperspace(Type: CI, Name: productIndex, LogVersion: 1) [name#1249,date#1251,qty#1250] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://default@honghaigen2.dfs.core.windows.net/hyperspace/indexes-6785343/pro..., PartitionFilters: [], PushedFilters: [IsNotNull(qty), GreaterThan(qty,10), IsNotNull(name), EqualTo(name,banana)], ReadSchema: struct , SelectedBucketsCount: 1 out of 200 ============================================================= Plan without indexes: ============================================================= SortMergeJoin [name#1225], [name#1249], Inner :- *(2) Sort [name#1225 ASC NULLS FIRST], false, 0 : +- Exchange hashpartitioning(name#1225, 200), [id=#499] : +- *(1) Project [name#1225, qty#1226, date#1227] : +- *(1) Filter (isnotnull(name#1225) && (name#1225 = banana)) : +- *(1) FileScan parquet [name#1225,qty#1226,date#1227] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://default@honghaigen2.dfs.core.windows.net/hyperspace/data-6785343/produc..., PartitionFilters: [], PushedFilters: [IsNotNull(name), EqualTo(name,banana)], ReadSchema: struct +- *(4) Sort [name#1249 ASC NULLS FIRST], false, 0 +- Exchange hashpartitioning(name#1249, 200), [id=#505] +- *(3) Project [name#1249, qty#1250, date#1251] +- *(3) Filter (((isnotnull(qty#1250) && (qty#1250 > 10)) && isnotnull(name#1249)) && (name#1249 = banana)) +- *(3) FileScan parquet [name#1249,qty#1250,date#1251] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://default@honghaigen2.dfs.core.windows.net/hyperspace/data-6785343/produc..., PartitionFilters: [], PushedFilters: [IsNotNull(qty), GreaterThan(qty,10), IsNotNull(name), EqualTo(name,banana)], ReadSchema: struct ============================================================= Indexes used: ============================================================= productIndex:abfss://default@honghaigen2.dfs.core.windows.net/hyperspace/indexes-6785343/productIndex/v__=0 ============================================================= Physical operator stats: ============================================================= +-------------------------------------------------------------+-------------------+------------------+----------+ | Physical Operator|Hyperspace Disabled|Hyperspace Enabled|Difference| +-------------------------------------------------------------+-------------------+------------------+----------+ | *InputAdapter| 4| 2| -2| |*Scan Hyperspace(Type: CI, Name: productIndex, LogVersion: 1)| 0| 2| 2| | *Scan parquet| 2| 0| -2| | *ShuffleExchange| 2| 0| -2| | *Sort| 2| 0| -2| | *WholeStageCodegen| 5| 3| -2| | Filter| 2| 2| 0| | Project| 2| 2| 0| | SortMergeJoin| 1| 1| 0| +-------------------------------------------------------------+-------------------+------------------+----------+

In [24]:
// Append new files.
var appendProducts = new List<GenericRow>()
{
    new GenericRow(new object[] {"orange", 13, "2020-11-01"}),
    new GenericRow(new object[] {"banana", 5, "2020-11-01"})
};
    
DataFrame appendData = spark.CreateDataFrame(appendProducts, productsSchema);
appendData.Write().Mode("Append").Parquet(testDataLocation);

StatementMeta(medium, 21, 24, Finished, Available)

Hybrid scan is disabled by default. Therefore, you will see that since we appended new data, Hyperspace will decide NOT to use the index.

In the output, you will see no plan differences (hence no highlighting).

In [25]:
// Hybrid Scan configs are false by default.
spark.Conf().Set("spark.hyperspace.index.hybridscan.enabled", "false");
spark.Conf().Set("spark.hyperspace.index.hybridscan.delete.enabled", "false");

DataFrame testDFWithAppend = spark.Read().Parquet(testDataLocation);
DataFrame filter1 = testDFWithAppend.Filter("name = 'banana'");
DataFrame filter2 = testDFWithAppend.Filter("qty > 10");
DataFrame query = filter1.Join(filter2, filter1.Col("name") == filter2.Col("name"));

query.Show();

hyperspace.Explain(query, true, input => DisplayHTML(input));

StatementMeta(medium, 21, 25, Finished, Available)

+------+---+----------+------+---+----------+
|  name|qty|      date|  name|qty|      date|
+------+---+----------+------+---+----------+
|banana|  1|2020-10-01|banana| 11|2020-10-03|
|banana|  5|2020-11-01|banana| 11|2020-10-03|
|banana| 11|2020-10-03|banana| 11|2020-10-03|
+------+---+----------+------+---+----------+



============================================================= Plan with indexes: ============================================================= SortMergeJoin [name#1408], [name#1414], Inner :- *(2) Sort [name#1408 ASC NULLS FIRST], false, 0 : +- Exchange hashpartitioning(name#1408, 200), [id=#682] : +- *(1) Project [name#1408, qty#1409, date#1410] : +- *(1) Filter (isnotnull(name#1408) && (name#1408 = banana)) : +- *(1) FileScan parquet [name#1408,qty#1409,date#1410] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://default@honghaigen2.dfs.core.windows.net/hyperspace/data-6785343/produc..., PartitionFilters: [], PushedFilters: [IsNotNull(name), EqualTo(name,banana)], ReadSchema: struct +- *(4) Sort [name#1414 ASC NULLS FIRST], false, 0 +- Exchange hashpartitioning(name#1414, 200), [id=#688] +- *(3) Project [name#1414, qty#1415, date#1416] +- *(3) Filter (((isnotnull(qty#1415) && (qty#1415 > 10)) && (name#1414 = banana)) && isnotnull(name#1414)) +- *(3) FileScan parquet [name#1414,qty#1415,date#1416] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://default@honghaigen2.dfs.core.windows.net/hyperspace/data-6785343/produc..., PartitionFilters: [], PushedFilters: [IsNotNull(qty), GreaterThan(qty,10), EqualTo(name,banana), IsNotNull(name)], ReadSchema: struct ============================================================= Plan without indexes: ============================================================= SortMergeJoin [name#1408], [name#1414], Inner :- *(2) Sort [name#1408 ASC NULLS FIRST], false, 0 : +- Exchange hashpartitioning(name#1408, 200), [id=#634] : +- *(1) Project [name#1408, qty#1409, date#1410] : +- *(1) Filter (isnotnull(name#1408) && (name#1408 = banana)) : +- *(1) FileScan parquet [name#1408,qty#1409,date#1410] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://default@honghaigen2.dfs.core.windows.net/hyperspace/data-6785343/produc..., PartitionFilters: [], PushedFilters: [IsNotNull(name), EqualTo(name,banana)], ReadSchema: struct +- *(4) Sort [name#1414 ASC NULLS FIRST], false, 0 +- Exchange hashpartitioning(name#1414, 200), [id=#640] +- *(3) Project [name#1414, qty#1415, date#1416] +- *(3) Filter (((isnotnull(qty#1415) && (qty#1415 > 10)) && (name#1414 = banana)) && isnotnull(name#1414)) +- *(3) FileScan parquet [name#1414,qty#1415,date#1416] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://default@honghaigen2.dfs.core.windows.net/hyperspace/data-6785343/produc..., PartitionFilters: [], PushedFilters: [IsNotNull(qty), GreaterThan(qty,10), EqualTo(name,banana), IsNotNull(name)], ReadSchema: struct ============================================================= Indexes used: ============================================================= ============================================================= Physical operator stats: ============================================================= +-----------------+-------------------+------------------+----------+ |Physical Operator|Hyperspace Disabled|Hyperspace Enabled|Difference| +-----------------+-------------------+------------------+----------+ | Filter| 2| 2| 0| | InputAdapter| 4| 4| 0| | Project| 2| 2| 0| | Scan parquet| 2| 2| 0| | ShuffleExchange| 2| 2| 0| | Sort| 2| 2| 0| | SortMergeJoin| 1| 1| 0| |WholeStageCodegen| 5| 5| 0| +-----------------+-------------------+------------------+----------+

### Enable Hybrid Scan

In plan with indexes, you can see
`Exchange hashpartitioning` required only for appended files so that we could still utilize the "shuffled" index data with appended files. `BucketUnion` is used to merge "shuffled" appended files with the index data.

In [26]:
// Enable Hybrid Scan config. "delete" config is not necessary.
spark.Conf().Set("spark.hyperspace.index.hybridscan.enabled", "true");
// spark.Conf().Set("spark.hyperspace.index.hybridscan.delete.enabled", "true");
spark.EnableHyperspace();
// Need to redefine query to recalculate the query plan.
DataFrame query = filter1.Join(filter2, filter1.Col("name") == filter2.Col("name"));

query.Show();

hyperspace.Explain(query, true, input => DisplayHTML(input));

StatementMeta(medium, 21, 26, Finished, Available)

+------+---+----------+------+---+----------+
|  name|qty|      date|  name|qty|      date|
+------+---+----------+------+---+----------+
|banana| 11|2020-10-03|banana| 11|2020-10-03|
|banana|  1|2020-10-01|banana| 11|2020-10-03|
|banana|  5|2020-11-01|banana| 11|2020-10-03|
+------+---+----------+------+---+----------+



============================================================= Plan with indexes: ============================================================= SortMergeJoin [name#1408], [name#1564], Inner :- *(3) Sort [name#1408 ASC NULLS FIRST], false, 0 : +- BucketUnion 200 buckets, bucket columns: [name] : :- *(1) Project [name#1408, qty#1409, date#1410] : : +- *(1) Filter (isnotnull(name#1408) && (name#1408 = banana)) : : +- *(1) FileScan Hyperspace(Type: CI, Name: productIndex, LogVersion: 1) [name#1408,date#1410,qty#1409] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://default@honghaigen2.dfs.core.windows.net/hyperspace/indexes-6785343/pro..., PartitionFilters: [], PushedFilters: [IsNotNull(name), EqualTo(name,banana)], ReadSchema: struct , SelectedBucketsCount: 1 out of 200 : +- Exchange hashpartitioning(name#1408, 200), [id=#886] : +- *(2) Project [name#1408, qty#1409, date#1410] : +- *(2) Filter (isnotnull(name#1408) && (name#1408 = banana)) : +- *(2) FileScan parquet [name#1408,qty#1409,date#1410] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://default@honghaigen2.dfs.core.windows.net/hyperspace/data-6785343/produc..., PartitionFilters: [], PushedFilters: [IsNotNull(name), EqualTo(name,banana)], ReadSchema: struct +- *(6) Sort [name#1564 ASC NULLS FIRST], false, 0 +- BucketUnion 200 buckets, bucket columns: [name] :- *(4) Project [name#1564, qty#1565, date#1566] : +- *(4) Filter (((isnotnull(qty#1565) && (qty#1565 > 10)) && isnotnull(name#1564)) && (name#1564 = banana)) : +- *(4) FileScan Hyperspace(Type: CI, Name: productIndex, LogVersion: 1) [name#1564,date#1566,qty#1565] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://default@honghaigen2.dfs.core.windows.net/hyperspace/indexes-6785343/pro..., PartitionFilters: [], PushedFilters: [IsNotNull(qty), GreaterThan(qty,10), IsNotNull(name), EqualTo(name,banana)], ReadSchema: struct , SelectedBucketsCount: 1 out of 200 +- Exchange hashpartitioning(name#1564, 200), [id=#894] +- *(5) Project [name#1564, qty#1565, date#1566] +- *(5) Filter (((isnotnull(qty#1565) && (qty#1565 > 10)) && isnotnull(name#1564)) && (name#1564 = banana)) +- *(5) FileScan parquet [name#1564,qty#1565,date#1566] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://default@honghaigen2.dfs.core.windows.net/hyperspace/data-6785343/produc..., PartitionFilters: [], PushedFilters: [IsNotNull(qty), GreaterThan(qty,10), IsNotNull(name), EqualTo(name,banana)], ReadSchema: struct ============================================================= Plan without indexes: ============================================================= SortMergeJoin [name#1408], [name#1564], Inner :- *(2) Sort [name#1408 ASC NULLS FIRST], false, 0 : +- Exchange hashpartitioning(name#1408, 200), [id=#817] : +- *(1) Project [name#1408, qty#1409, date#1410] : +- *(1) Filter (isnotnull(name#1408) && (name#1408 = banana)) : +- *(1) FileScan parquet [name#1408,qty#1409,date#1410] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://default@honghaigen2.dfs.core.windows.net/hyperspace/data-6785343/produc..., PartitionFilters: [], PushedFilters: [IsNotNull(name), EqualTo(name,banana)], ReadSchema: struct +- *(4) Sort [name#1564 ASC NULLS FIRST], false, 0 +- Exchange hashpartitioning(name#1564, 200), [id=#823] +- *(3) Project [name#1564, qty#1565, date#1566] +- *(3) Filter (((isnotnull(qty#1565) && (qty#1565 > 10)) && isnotnull(name#1564)) && (name#1564 = banana)) +- *(3) FileScan parquet [name#1564,qty#1565,date#1566] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://default@honghaigen2.dfs.core.windows.net/hyperspace/data-6785343/produc..., PartitionFilters: [], PushedFilters: [IsNotNull(qty), GreaterThan(qty,10), IsNotNull(name), EqualTo(name,banana)], ReadSchema: struct ============================================================= Indexes used: ============================================================= productIndex:abfss://default@honghaigen2.dfs.core.wi

### Cleanup
To make this notebook self-contained and not leave any dangling data, we have some small clean-up code below. 

In [27]:
using Microsoft.Spark.Extensions.Azure.Synapse.Analytics.Notebook.MSSparkUtils;

FS.Rm(dataPath, true);
FS.Rm(indexLocation, true);

StatementMeta(medium, 21, 27, Finished, Available)